In [1]:
pwd

'/Users/vmuccion/Documents/Projects/ClimateEducation/cgpt_classification'

In [2]:
import pandas as pd
import os
import torch
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, EarlyStoppingCallback, Trainer, TrainingArguments, AutoModelForSequenceClassification, AutoModelForPreTraining
from collections import OrderedDict
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from transformers import pipeline
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.utils import shuffle

/Users/vmuccion/opt/anaconda3/envs/ClimateEducation/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-02-09 16:26:19.573835: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
os.environ["WANDB_DISABLED"] = "true"

In [8]:
path = "../Data/"

#r"C:\Saeid\Prj100\SA_45_DSS_embrace"
train_file = "seen1.xlsx"
test_file = "unseen.xlsx"
train_path = os.path.join(path, train_file)
test_path = os.path.join(path, test_file)
#cgpt_path = r"C:\Saeid\Prj100\SA_45_DSS_embrace\models\climateGPT2_ready_RL\pytorch_model.bin" # the path to bin file of cgpt((this is ckeygpt do we want cgpt or this?))
#output_path = r"C:\Saeid\Prj100\SA_45_DSS_embrace\relevancy_cgpt_save"

cgpt_path = "/Users/vmuccion/Documents/Projects/ClimateEducation/cgpt_classification/models/climateGPT2_ready_RL/pytorch_model.bin"
output_path = "/Users/vmuccion/Documents/Projects/ClimateEducation/cgpt_classification/relevancy_cgpt_save/"

In [9]:
#set_epoch = 100
set_epoch = 2
train_data = pd.read_excel(train_path)
print(train_data.head())

test_data = pd.read_excel(test_path)
print(test_data.head())

train_data.describe()

test_data.describe()

ready_to_train = train_data[["Abstract", "Topic"]]
print(ready_to_train.head())

train, test = train_test_split(train_data[["Abstract", "Relevant"]], test_size=0.33, random_state=42)

print(train.head())
print(test.head())
class relevancyDataset(Dataset):

    def __init__(self, data, tokenizer):
        x = data["Abstract"].tolist()
        label = data["Relevant"].tolist()

        self.tokenizer = tokenizer
        self.x = x
        self.label = label

    # ---------------------------------------------#

    def __len__(self):
        return len(self.label)

    # ---------------------------------------------#

    def __getitem__(self, i):
        encodings_dict = tokenizer(self.x[i],
                                   truncation=True,
                                   max_length=1500,
                                   padding="max_length")

        input_ids = encodings_dict['input_ids']
        attention_mask = encodings_dict['attention_mask']

        return {'label':torch.tensor(self.label[i]),
            'input_ids': torch.tensor(input_ids),
            'attention_mask': torch.tensor(attention_mask)}


                           DOI  \
0   10.1016/j.nedt.2021.105144   
1   10.1007/s11422-021-10080-6   
2     10.1016/j.pt.2010.06.009   
3            10.1111/jnu.12326   
4  10.1016/j.jsurg.2020.11.001   

                                               Title  \
0  A mixed method study on global warming, climat...   
1  Global challenges need attention now: educatin...   
2  Soil-transmitted helminthiases: implications o...   
3  Climate Change, Climate Justice, and Environme...   
4  The Role of Preoperative Briefing and Postoper...   

                                            Abstract  \
0  AIM: This study aims to evaluate the knowledge...   
1  This Editorial sets the stage for 18 papers on...   
2  Soil-transmitted helminthiases (STHs) collecti...   
3  PURPOSE: Climate change is an emerging challen...   
4  OBJECTIVE: To study the impact of a new preope...   

                            Source title  PubYear  \
0                  Nurse Education Today     2021   
1  Cultural Stu

In [ ]:
base_model = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(base_model)
tokenizer.pad_token = tokenizer.eos_token

tt = GPT2Tokenizer.from_pretrained(base_model)
tt.pad_token = tt.eos_token

train_data_set = relevancyDataset(train, tokenizer)
test_data_set = relevancyDataset(test, tokenizer)


model_final = AutoModelForSequenceClassification.from_pretrained(output_path)
model_final.config.pad_token_id = model_final.config.eos_token_id
xd = []
dx = []
for k, i in enumerate(test_data["Abstract"]):
    print(str(k)+"/"+str(len(test_data["Abstract"])))
    inputs = tokenizer(i,truncation=True,max_length=750,padding="max_length", return_tensors='pt')
    logits = model_final(**inputs).logits

    softmax= torch.nn.Softmax()

    predicted_class_id = logits.argmax().item()
    prcnt = round(softmax(logits).max().item()*100, 2)


    xd.append(predicted_class_id)
    dx.append(prcnt)
req = pd.DataFrame({"Abstract":test_data["Abstract"], "Relevant": xd, "DOI":test_data["DOI"], "Title":test_data["Title"], "Source title":test_data["Source title"],"PubYear":test_data["PubYear"],"Authors":test_data["Authors"],"Times cited":test_data["Times cited"],"Probability":dx})
print(req.head())

In [7]:
ff = []
for k, i in enumerate(train_data["Abstract"]):
    print(str(k)+"/"+str(len(train_data["Abstract"])))
    ff.append(100.0)
)
qer = pd.DataFrame({"Abstract":train_data["Abstract"], "Relevant": train_data["Relevant"], "DOI":train_data["DOI"], "Title":train_data["Title"], "Source title":train_data["Source title"],"PubYear":train_data["PubYear"],"Authors":train_data["Authors"],"Times cited":train_data["Times cited"],"Probability":ff})

print("END!!")
print(qer.head())

result = pd.concat([qer, req])
print(result.head())
rls = shuffle(result)
print(rls.head())
rls.to_excel("../Data/output_relevant_unseen_seen_merge_shuffle_Feb2023.xlsx")

ValueError: unable to parse C:\Saeid\Prj100\SA_45_DSS_embrace\relevancy_save\config.json as a URL or as a local path